# 第19章 马尔可夫链蒙特卡罗法    
     
    19.1 蒙特卡罗法    
        19.1.1 随机抽样    
        19.1.2 数学期望估计    
        19.1.3 积分计算    
    19.2 马尔可夫链    
        19.2.1 基本定义    
        19.2.2 离散状态马尔可夫链    
        19.2.3 连续状态马尔可夫链    
        19.2.4 马尔可夫链的性质    
    19.3 马尔可夫链蒙特卡罗法    
        19.3.1 基本想法    
        19.3.2 基本步骤    
        19.3.3 马尔可夫链蒙特卡罗法与统计学习    
    19.4 Metropolis-Hastings 算法    
        19.4.1 基本原理    
        19.4.2 Metropolis-Hastings 算法    
        19.4.3 单分量 Metropolis-Hastings 算法    
    19.5 吉布斯抽样    
        19.5.1 基本原理    
        19.5.2 吉布斯抽样算法    
        19.5.3 抽样计算  
    

In [1]:
import random
import math
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)

蒙特卡罗法（Monte Carlo method），也称为统计模拟方法（statistical simulation method），是通过从概率模型的随机抽样进行近似数值计算的方法。
马尔可夫链蒙特卡罗法（Markov Chain Monte Carlo, MCMC）,则是以马尔可夫链（Markov chain）为概率模型的蒙特卡罗法。
马尔可夫链蒙特卡罗法构建一个马尔可夫链，使其平稳分布就是要进行抽样的分布，首先基于该马尔可夫链进行随机游走，产生样本的序列，之后使用该平稳分布的样本进行近似数值计算。

Metropolis - Hastings 算法足最基本的马尔可夫链蒙特卡罗法，Metropolis等人在1953年提出原始的算法，Hastings在1970年对之加以推广，形成了现在的形式。
吉布斯抽样（Gibbs sampling）是更简单、使用更广泛的马尔可夫链蒙特卡罗法，1984年由S.Geman和D.Geman提出。

马尔可夫链蒙特卡罗法被应用于概率分布的估计、定积分的近似计算、最优化问题的近似求解等问题，特别是被应用于统计学习中概率模型的学习与推理，是重要的统计学习计算方法。

## 19.3 马尔可夫链蒙特卡罗法    

###     19.3.1 基本想法    

假设目标是对一个概率分布进行随机抽样，或者是求函数关于该概率分布的数学期望。
可以采用传统的蒙特卡罗法，如接受-拒绝法、重要性抽样法，也可以使用马尔可夫链蒙特卡罗法。
马尔可夫链蒙特卡罗法更适合于随机变量是多元的、密度函数是非标准形式的、随机变量各分量不独立等情况。

假设多元随机变量$x$，满足$x \in \mathcal{X}$，其概率密度函数为 $p(x)$，$f(x)$ 为定义在$x \in \mathcal{X}$上的函数，目标是获得概率分布 $p(x)$ 的样本集合，以及求函 数 $f(x)$ 的数学期望 $E_{p(x)}[f(x)]$。

#### 马尔可夫链蒙特卡罗法的基本想法

在随机变量$x$的状态空间$S$上定义一个满足遍历定理的马尔可夫链 $X = \{ X_0, X_1, \cdots, X_t, \cdots \}$ ，使其平稳分布就是抽样的目标分布 $p(x)$。
然后在这个马尔可夫链上进行随机游走，每个时刻得到一个样本。
根据遍历定理，当时间趋于无穷时，样本的分布趋近平稳分布，样本的函数均值趋近函数的数学期望。
所以，当时间足够长时（时刻大于某个正整数 $m$）,在之后的时间（时刻小于等于某个正整数$n$ , $n > m$）里随机游走得到的样本集合 $\{x_{m+1},x_{m+2},\cdots, x_n \}$ 就 是目标概率分布的抽样结果，得到的函数均值（遍历均值）就是要计算的数学期望值：

$$ \hat{E} f=\frac{1}{n-m} \sum_{i=m+1}^{n} f\left(x_{i}\right) $$

到时刻 $m$ 为止的时间段称为**燃烧期**。

**如何构建具体的马尔可夫链成为这个方法的关键**

连续变量的时候，需要定义转移核函数；
离散变量的时候，需要定义转移矩阵。
一个方法是定义特殊的转移核函数或者转移矩阵，构建可逆马尔可夫链，这样可以保证遍历定理成立。
常用的马尔可夫链蒙特卡罗法有 **Metropolis-Hastings算法**、**吉布斯抽样**。

由于这个马尔可夫链满足遍历定理，随机游走的起始点并不影响得到的结果，即从不同的起始点出发，都会收敛到同一平稳分布。

马尔可夫链蒙特卡罗法的收敛性的判断通常是经验性的。
比如，在马尔可夫链上进行随机游走，检验遍历均值是否收敛。
具体地，每隔一段时间取一次样本，得到多个样本以后，计算遍历均值，当计筇的均值稳定 后，认为马尔可夫链己经收敛。
再比如，在马尔可夫链上并行进行多个随机游走，比较各个随机游走的遍历均值是否接近一致。

马尔可夫链蒙特卡罗法中得到的样本序列，相邻的样本点是相关的，而不是独立的。
因此，在需要独立样本时，可以在该样本序列中再次进行随机抽样，比如每隔一段时间取一次样本，将这样得到的子样本集合作为独立样本集合。

马尔可夫链蒙特卡罗法比**接受-拒绝法**更容易实现，因为只需要定义马尔可夫链，而不需要定义建议分布。
一般来说马尔可夫链蒙特卡罗法比**接受-拒绝法**效率更高，没有大量被拒绝的样本，虽然**燃烧期**的样本也要抛弃。

#### 几个重要问题
* 1. 如何定义马尔可夫链，保证马尔可夫链蒙特卡罗法的条件成立.
* 2. 如何确定收敛步数 $m$，保证样本抽样的无偏性。
* 3. 如何确定迭代步数 $n$，保证遍历均值计算的精度。

###     19.3.2 基本步骤    

* 1. 在随机变量 $x$ 的状态空间 $\mathcal{S}$ 上构造一个满足遍历定理的马尔可夫链，使其平稳分布为目标分布 $p(x)$;
* 2. 从状态空间的某一点 $x_0$ 出发，用构造的马尔可夫链进行随机游走，产生样本序列 $x_0, x_1, \cdots, x_t, \cdots $；
* 3. 应用马尔可夫链的遍历定理，确定正整数 $m$ 和 $n$，$(m<n)$， 得到样本集合 $\{x_{m+1},x_{m+2},\cdots, x_n \}$，求得函数 $f(x)$ 的均值（遍历均值）

$$ \hat{E} f=\frac{1}{n-m} \sum_{i=m+1}^{n} f\left(x_{i}\right) $$

###     19.3.3 马尔可夫链蒙特卡罗法与统计学习    